In [21]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [22]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [23]:
%cd /content/gdrive/MyDrive/Black Coffer

/content/gdrive/MyDrive/Black Coffer


In [24]:
df1 = pd.read_excel('Input.xlsx')
df1.head()

,URL_ID,URL
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...
1,38.0,https://insights.blackcoffer.com/what-if-the-c...
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...
3,40.0,https://insights.blackcoffer.com/will-machine-...
4,41.0,https://insights.blackcoffer.com/will-ai-repla...


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os


# Creating a directory to save the text files
if not os.path.exists('article_texts'):
    os.makedirs('article_texts')

# going over each URL over each URL
for URL in df1['URL']:
    # Replacing forward slashes with an underscore in the file name so that we can save it.
    file_name = URL.replace('/', '_') + '.txt'
    file_path = os.path.join('article_texts', file_name)
    
    # requesting to the URL
    response = requests.get(URL)
    html = response.text
    
    # Using BeautifulSoup to extract articles text
    soup = BeautifulSoup(html, 'html.parser')
    article = soup.find('article')
    if article:
        article_text = article.get_text()
    else:
        article_text = soup.get_text()
    
    # Saving the extracted article text to a new file with URL as its name
    with open(file_path, 'w') as f:
        f.write(article_text)

In [31]:
import os
import nltk
from textblob import TextBlob

nltk.download('punkt')

results = []

# Reading the positive and negative words from local text files
with open("positive-words.txt", "r", encoding='ISO-8859-1') as positive_file:
    positive_words = set(positive_file.read().splitlines())

with open("negative-words.txt", "r", encoding='ISO-8859-1') as negative_file:
    negative_words = set(negative_file.read().splitlines())

# Going over text file in the saved article dir:
for file_name in os.listdir('article_texts'):
    file_path = os.path.join('article_texts', file_name)
    with open(file_path, 'r') as f:
        text = f.read()

    # TextBlob object
    blob = TextBlob(text)

    # Word Tokenizing
    tokens = nltk.word_tokenize(str(blob))

    # Positive Score
    positive_score = 0
    for word in tokens:
        if word in positive_words:
            positive_score += 1

    # Negative
    negative_score = 0
    for word in tokens:
        if word in negative_words:
            negative_score += 1

    # Calculate Polarity Score
    polarity_score = (positive_score - negative_score) / (positive_score + negative_score + 0.000001)

    # Calculate Subjectivity Score
    total_words_count = len(tokens)
    subjectivity_score = (positive_score + negative_score) / (total_words_count + 0.000001)

    # Variables
    avg_sentence_length = sum(len(sentence.words) for sentence in blob.sentences) / len(blob.sentences)
    complex_word_count = sum(1 for word in blob.words if len(word) >= 7)
    total_word_count = len(blob.words)
    percent_complex_words = complex_word_count / total_word_count * 100
    avg_words_per_sentence = avg_sentence_length
    fog_index = 0.4 * (avg_words_per_sentence + percent_complex_words)

    personal_pronouns = sum(1 for word in blob.words if word in ['I', 'me', 'my', 'mine', 'we', 'us', 'our', 'ours'])
    avg_word_length = sum(len(word) for word in blob.words) / total_word_count

    # Append the results to the list
    results.append([positive_score, negative_score, polarity_score, subjectivity_score, avg_sentence_length,
                    percent_complex_words, fog_index, avg_words_per_sentence, complex_word_count, total_word_count,
                    personal_pronouns, avg_word_length])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [32]:
# Create a DataFrame from the results list and write it to an CSV file
df2 = pd.DataFrame(results, columns=['POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE', 'SUBJECTIVITY SCORE',
                                    'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX',
                                    'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT',
                                    'PERSONAL PRONOUNS', 'AVG WORD LENGTH'])

In [33]:
df1.head()

,URL_ID,URL
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...
1,38.0,https://insights.blackcoffer.com/what-if-the-c...
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...
3,40.0,https://insights.blackcoffer.com/will-machine-...
4,41.0,https://insights.blackcoffer.com/will-ai-repla...


In [34]:
df2.head()

,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,PERSONAL PRONOUNS,AVG WORD LENGTH
0,77,36,0.362832,0.029086,29.468354,43.685567,29.261569,29.468354,1017,2328,4,7.185137
1,67,41,0.240741,0.030647,22.860465,33.316378,22.470737,22.860465,655,1966,8,6.805697
2,73,38,0.315315,0.029149,25.816092,41.718611,27.013881,25.816092,937,2246,2,7.052093
3,73,29,0.431373,0.027486,22.484536,33.745988,22.492210,22.484536,736,2181,22,6.655663
4,59,28,0.356322,0.022498,27.927711,35.418464,25.338470,27.927711,821,2318,14,6.701467


In [35]:
df2.insert(0, "URL", df1['URL'], True)

In [36]:
df2.to_csv('output.csv', index=False)

In [37]:
df2.head()

,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,PERSONAL PRONOUNS,AVG WORD LENGTH
0,https://insights.blackcoffer.com/ai-in-healthc...,77,36,0.362832,0.029086,29.468354,43.685567,29.261569,29.468354,1017,2328,4,7.185137
1,https://insights.blackcoffer.com/what-if-the-c...,67,41,0.240741,0.030647,22.860465,33.316378,22.470737,22.860465,655,1966,8,6.805697
2,https://insights.blackcoffer.com/what-jobs-wil...,73,38,0.315315,0.029149,25.816092,41.718611,27.013881,25.816092,937,2246,2,7.052093
3,https://insights.blackcoffer.com/will-machine-...,73,29,0.431373,0.027486,22.484536,33.745988,22.492210,22.484536,736,2181,22,6.655663
4,https://insights.blackcoffer.com/will-ai-repla...,59,28,0.356322,0.022498,27.927711,35.418464,25.338470,27.927711,821,2318,14,6.701467
